In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define test image paths
test_wf_path = '/content/drive/MyDrive/BioSR/Test data/CCPs/test_wf'
test_gt_path = '/content/drive/MyDrive/BioSR/Test data/CCPs/test_gt'
output_dir = '/content/drive/MyDrive/BioSR/Test data/CCPs/output'

# Create output directory if not exists
os.makedirs(output_dir, exist_ok=True)


Mounted at /content/drive


In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import os

# Define image loading function for specific folders
def load_images_from_selected_folders(base_folder, selected_folders):
    images = []
    filenames = []
    for folder in selected_folders:
        folder_path = os.path.join(base_folder, folder)
        for filename in os.listdir(folder_path):
            if filename.endswith('.tif'):
                img_path = os.path.join(folder_path, filename)
                img = Image.open(img_path).convert('L')  # Convert to grayscale
                img_tensor = transforms.ToTensor()(img)  # Convert image to tensor
                images.append(img_tensor)
                filenames.append(filename)
                print(f"Loading image: {img_path}")
    return images, filenames

# Define folders to load
selected_folders = ['level_01', 'level_05', 'level_09']

# Load test WF images
test_wf_images, test_wf_filenames = load_images_from_selected_folders(test_wf_path, selected_folders)

# Load test GT images
test_gt_images, test_gt_filenames = load_images_from_selected_folders(test_gt_path, [''])  # GT folder has no subfolders


Loading image: /content/drive/MyDrive/BioSR/Test data/CCPs/test_wf/level_01/001.tif
Loading image: /content/drive/MyDrive/BioSR/Test data/CCPs/test_wf/level_01/002.tif
Loading image: /content/drive/MyDrive/BioSR/Test data/CCPs/test_wf/level_01/003.tif
Loading image: /content/drive/MyDrive/BioSR/Test data/CCPs/test_wf/level_01/004.tif
Loading image: /content/drive/MyDrive/BioSR/Test data/CCPs/test_wf/level_01/005.tif
Loading image: /content/drive/MyDrive/BioSR/Test data/CCPs/test_wf/level_01/006.tif
Loading image: /content/drive/MyDrive/BioSR/Test data/CCPs/test_wf/level_01/007.tif
Loading image: /content/drive/MyDrive/BioSR/Test data/CCPs/test_wf/level_01/008.tif
Loading image: /content/drive/MyDrive/BioSR/Test data/CCPs/test_wf/level_01/009.tif
Loading image: /content/drive/MyDrive/BioSR/Test data/CCPs/test_wf/level_01/010.tif
Loading image: /content/drive/MyDrive/BioSR/Test data/CCPs/test_wf/level_01/011.tif
Loading image: /content/drive/MyDrive/BioSR/Test data/CCPs/test_wf/level_01/

In [ ]:
!git clone https://github.com/JingyunLiang/SwinIR.git
!pip install timm

Cloning into 'SwinIR'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 333 (delta 6), reused 5 (delta 2), pack-reused 320 (from 1)
Receiving objects: 100% (333/333), 29.84 MiB | 31.24 MiB/s, done.
Resolving deltas: 100% (119/119), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.1 MB/s eta 0:00:00


In [ ]:
import torch
from SwinIR.models.network_swinir import SwinIR

# Load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SwinIR(
    upscale=2,  # Upscaling factor
    in_chans=1,  # Grayscale input
    img_size=32,  # Input image size
    window_size=8,  # Window size
    depths=[4, 4, 4, 4],  # Layer depths
    embed_dim=120,  # Embedding dimension
    num_heads=[4, 4, 4, 4],  # Number of attention heads
    mlp_ratio=2,  # MLP ratio
    upsampler='pixelshuffledirect',  # Upsampler
    resi_connection='1conv'  # Residual connection
).to(device)

# Load pre-trained model weights
model.load_state_dict(torch.load('/content/drive/MyDrive/BioSR/CCP/swinir_best_model.pth', map_location=device))
model.eval()


<ipython-input-32-5ca987ab414b>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/BioSR/CCP/swinir_best_model.pth', 

SwinIR(
  (conv_first): Conv2d(1, 120, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (patch_embed): PatchEmbed(
    (norm): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
  )
  (patch_unembed): PatchUnEmbed()
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): RSTB(
      (residual_group): BasicLayer(
        dim=120, input_resolution=(32, 32), depth=4
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            dim=120, input_resolution=(32, 32), num_heads=4, window_size=8, shift_size=0, mlp_ratio=2
            (norm1): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              dim=120, window_size=(8, 8), num_heads=4
              (qkv): Linear(in_features=120, out_features=360, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=120, out_features=120, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
       

In [ ]:
import torch
import os
import torchvision.utils as vutils

# Define normalization function
def normalize(img):
    return (img - img.min()) / (img.max() - img.min())

# Directory to save output images
output_dir = "/content/drive/MyDrive/BioSR/Test data/CCPs/output"  # Replace with your output directory path

# Ensure directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process and save output images
for i, wf_img in enumerate(test_wf_images[:len(test_gt_images)]):  # Ensure WF and GT images length match
    wf_img = normalize(wf_img).to(device)  # Normalize and move to GPU

    # Perform inference with the model
    with torch.no_grad():  # Disable gradient calculation for inference
        output = model(wf_img.unsqueeze(0))  # Add batch dimension
        output_img = output.squeeze(0).cpu()  # Remove batch dimension and move to CPU

    # Convert the output image to grayscale (single-channel)
    if output_img.ndim == 3:  # If the output has more than one channel
        output_img = output_img.mean(dim=0, keepdim=True)  # Take the mean across channels to convert to grayscale

    # Remove the extension if it exists in the filename
    base_filename = test_wf_filenames[i].replace('.tif', '')  # Remove '.tif' if it exists

    # Save the output image in grayscale
    output_img_path = os.path.join(output_dir, f"{base_filename}.tif")  # Save in .tif format
    vutils.save_image(output_img, output_img_path, normalize=True)  # Save the image
    print(f"Saved output: {output_img_path}")

    # Free up memory
    del wf_img, output, output_img  # Delete variables to free GPU memory
    torch.cuda.empty_cache()  # Clear the cache

print("All images processed and saved.")


Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/001.tif
Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/002.tif
Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/003.tif
Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/004.tif
Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/005.tif
Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/006.tif
Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/007.tif
Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/008.tif
Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/009.tif
Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/010.tif
Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/011.tif
Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/012.tif
Saved output: /content/drive/MyDrive/BioSR/Test data/CCPs/output/013.tif
Saved output: /content/drive/MyDrive/BioSR/Test dat

In [ ]:
import os
import tifffile as tiff
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import numpy as np

# Directories for output and ground truth
output_dir = "/content/drive/MyDrive/BioSR/Test data/CCPs/output/"  # Ensure this is the correct directory path
gt_dir = "/content/drive/MyDrive/BioSR/Test data/CCPs/test_gt/"    # Ensure this is the correct directory path

# Retrieve actual filenames in the output folder and sort them
output_filenames = sorted([f for f in os.listdir(output_dir) if f.endswith('.tif')])

# Retrieve actual filenames in the ground truth folder and sort them
gt_filenames = sorted([f for f in os.listdir(gt_dir) if f.endswith('.tif')])

# Initialize lists to hold PSNR and SSIM values
psnr_values = []
ssim_values = []

# Check if the number of output and ground truth images are the same
if len(output_filenames) != len(gt_filenames):
    raise ValueError("Mismatch in the number of output and ground truth images!")

# Iterate through output filenames and calculate PSNR and SSIM
for i, output_filename in enumerate(output_filenames):
    # Construct the output image path
    output_img_path = os.path.join(output_dir, output_filename)

    # Construct the ground truth image path using the corresponding index
    gt_img_path = os.path.join(gt_dir, gt_filenames[i])

    # Read the images
    output_img = tiff.imread(output_img_path)
    gt_img = tiff.imread(gt_img_path)

    # Convert output image to grayscale if it has 3 channels
    if output_img.ndim == 3:  # Check if the image has 3 dimensions (channels)
        output_img = np.mean(output_img, axis=2)  # Convert to grayscale by averaging channels

    # Ensure both images have the same dimensions
    if gt_img.shape != output_img.shape:
        raise ValueError(f"Shape mismatch: Ground truth image {gt_img_path} and output image {output_img_path} have different shapes.")

    # Calculate PSNR and SSIM
    psnr_value = psnr(gt_img, output_img)
    ssim_value = ssim(gt_img, output_img, data_range=gt_img.max() - gt_img.min())

    psnr_values.append(psnr_value)
    ssim_values.append(ssim_value)

    print(f"Image {i+1}: PSNR = {psnr_value:.2f}, SSIM = {ssim_value:.4f}")

# Display average metrics
average_psnr = sum(psnr_values) / len(psnr_values)
average_ssim = sum(ssim_values) / len(ssim_values)

print(f"\nAverage PSNR: {average_psnr:.2f}")
print(f"Average SSIM: {average_ssim:.4f}")


<ipython-input-34-5d1c3616b13a>:46: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  psnr_value = psnr(gt_img, output_img)


Image 1: PSNR = 24.16, SSIM = 0.7248
Image 2: PSNR = 24.24, SSIM = 0.7805
Image 3: PSNR = 24.45, SSIM = 0.7666
Image 4: PSNR = 28.29, SSIM = 0.8213
Image 5: PSNR = 24.21, SSIM = 0.7916
Image 6: PSNR = 25.36, SSIM = 0.7939
Image 7: PSNR = 26.91, SSIM = 0.8243
Image 8: PSNR = 28.39, SSIM = 0.8148
Image 9: PSNR = 25.15, SSIM = 0.7468
Image 10: PSNR = 27.00, SSIM = 0.8411
Image 11: PSNR = 25.74, SSIM = 0.8030
Image 12: PSNR = 25.84, SSIM = 0.8120
Image 13: PSNR = 24.63, SSIM = 0.8334
Image 14: PSNR = 27.48, SSIM = 0.8561
Image 15: PSNR = 21.48, SSIM = 0.6992

Average PSNR: 25.56
Average SSIM: 0.7939
